In [2]:
import os
import glob
import csv
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from sklearn.neighbors import NearestNeighbors

# ====== 前処理・特徴抽出用 関数群 ======
def load_and_preprocess_images(image_dir, target_size=(224, 224)):
    """
    指定フォルダ内の tif 画像を読み込み、
    VGG-16 用の前処理を施した numpy 配列を返す。
    返り値: 形状 (N, 224, 224, 3) の numpy 配列
    """
    import glob
    image_paths = glob.glob(os.path.join(image_dir, "*.tif"))
    images = []

    for path in image_paths:
        # 画像読み込み (cv2 は BGR 順なので注意)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Could not load image: {path}")
            continue

        # RGB に変換
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # リサイズ
        img = cv2.resize(img, target_size)
        # array 化
        img = np.array(img, dtype=np.float32)
        images.append(img)

    images = np.array(images)
    # VGG16 用の前処理
    images = preprocess_input(images)

    return images

def sample_images(images, n):
    """
    画像群 (images) からランダムに n 枚抽出して返す関数。
    images: shape = (N, 224, 224, 3)
    n: 抽出したい枚数
    """
    N = len(images)
    if N <= n:
        # 既に n 枚以下ならそのまま返す
        return images
    else:
        indices = np.random.choice(N, n, replace=False)
        return images[indices]

def extract_features(images, batch_size=32):
    """
    VGG-16 (fc2) の特徴ベクトルを抽出する。
    images: 形状 (N, 224, 224, 3)
    返り値: 形状 (N, 4096)
    """
    base_model = VGG16(include_top=True, weights='imagenet')
    # fc2層(4096次元)の出力を得るようにする
    model = Model(inputs=base_model.input, 
                  outputs=base_model.get_layer("fc2").output)
    
    features_list = []
    n_samples = images.shape[0]
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        batch = images[start_idx:end_idx]
        # 順伝搬
        batch_features = model.predict(batch, verbose=0)
        features_list.append(batch_features)
    
    features = np.concatenate(features_list, axis=0)
    return features

def compute_kth_radius(features, k=3):
    """
    与えられた特徴ベクトル集合に対して、
    各サンプルの「k番目の最近傍との距離」を返す。
    
    返り値: 半径 (radius) 配列, shape = (N,)
    """
    nbrs = NearestNeighbors(n_neighbors=k+1, metric='euclidean').fit(features)
    distances, indices = nbrs.kneighbors(features)
    # k番目の距離を取得 (distances[:, k])
    kth_distances = distances[:, k]
    return kth_distances

def is_in_manifold(sample, manifold_features, manifold_radius):
    """
    sample という特徴ベクトルが、manifold_features に属する
    いずれかのサンプルの超球 (半径 = manifold_radius[i]) に含まれるかどうかを返す。
    """
    diff = manifold_features - sample  # shape (N, D)
    dists = np.linalg.norm(diff, axis=1)  # shape (N,)
    within = dists <= manifold_radius
    return np.any(within)

def compute_precision_recall(features_r, features_g, k=3):
    """
    Precision と Recall を計算。
    features_r: real (original) 側の特徴ベクトル (shape = (Nr, D))
    features_g: generated (synthesis) 側の特徴ベクトル (shape = (Ng, D))
    """
    radius_r = compute_kth_radius(features_r, k)
    radius_g = compute_kth_radius(features_g, k)
    
    # Precision: 生成画像 (G) が R のマニフォールドに含まれる割合
    in_r_count = 0
    for g_j in features_g:
        if is_in_manifold(g_j, features_r, radius_r):
            in_r_count += 1
    precision = in_r_count / len(features_g)

    # Recall: 実画像 (R) が G のマニフォールドに含まれる割合
    in_g_count = 0
    for r_i in features_r:
        if is_in_manifold(r_i, features_g, radius_g):
            in_g_count += 1
    recall = in_g_count / len(features_r)
    
    return precision, recall


# ====== メイン部分 ======
if __name__ == "__main__":
    # 例: エポック 0, 10, 20, ..., 100 で評価すると仮定
    FIRST_EPOCH = 10
    LAST_EPOCH = 160
    EPOCH_INTERVAL = 10

    # 保存先 CSV ファイル名
    csv_file = "precision_recall.csv"

    # 1) CSV ファイルを初期化してヘッダー行を書き込む
    with open(csv_file, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "precision", "recall"])  # ヘッダー

    # 抽出したい画像枚数 (両フォルダとも 1500 枚ずつ)
    N_SAMPLE = 1500

    for epoch in range(FIRST_EPOCH, LAST_EPOCH + 10, EPOCH_INTERVAL):
        # === パラメータ設定 ===
        original_dir = "../dataset/basic"  # 実画像のフォルダ
        synthesis_dir = f"../synthesis_images/step6/epoch{epoch}"  # 生成画像のフォルダ
        k = 3                 # 近傍の個数
        batch_size = 32       # 特徴抽出時のバッチサイズ

        # === 画像読み込み & 前処理 ===
        original_images = load_and_preprocess_images(original_dir, target_size=(224, 224))
        synthesis_images = load_and_preprocess_images(synthesis_dir, target_size=(224, 224))

        # もしどちらかのフォルダに画像が 0 枚、あるいは 1500 枚未満ならスキップ
        if len(original_images) < N_SAMPLE or len(synthesis_images) < N_SAMPLE:
            print(f"[Epoch {epoch}] Not enough images in one of the folders (need at least {N_SAMPLE}).")
            continue

        # === それぞれ 1500 枚ずつランダムサンプリング ===
        original_images = sample_images(original_images, N_SAMPLE)
        synthesis_images = sample_images(synthesis_images, N_SAMPLE)

        # === 特徴ベクトル抽出 (VGG16 fc2) ===
        features_r = extract_features(original_images, batch_size=batch_size)
        features_g = extract_features(synthesis_images, batch_size=batch_size)

        # === Precision & Recall の計算 ===
        precision, recall = compute_precision_recall(features_r, features_g, k=k)

        # === コンソールに出力 (任意) ===
        print(f"[Epoch {epoch}] #original={len(original_images)}, #synthesis={len(synthesis_images)}")
        print(f"[Epoch {epoch}] Precision (k={k}): {precision:.4f}")
        print(f"[Epoch {epoch}] Recall    (k={k}): {recall:.4f}")

        # === CSV に追記 ===
        with open(csv_file, "a", newline='') as f:
            writer = csv.writer(f)
            writer.writerow([epoch, f"{precision:.4f}", f"{recall:.4f}"])


[Epoch 10] #original=1500, #synthesis=1500
[Epoch 10] Precision (k=3): 0.0000
[Epoch 10] Recall    (k=3): 0.0000
[Epoch 20] #original=1500, #synthesis=1500
[Epoch 20] Precision (k=3): 0.0000
[Epoch 20] Recall    (k=3): 0.0000
[Epoch 30] #original=1500, #synthesis=1500
[Epoch 30] Precision (k=3): 0.0000
[Epoch 30] Recall    (k=3): 0.0007
[Epoch 40] #original=1500, #synthesis=1500
[Epoch 40] Precision (k=3): 0.0173
[Epoch 40] Recall    (k=3): 0.0053
[Epoch 50] #original=1500, #synthesis=1500
[Epoch 50] Precision (k=3): 0.0833
[Epoch 50] Recall    (k=3): 0.0647
[Epoch 60] #original=1500, #synthesis=1500
[Epoch 60] Precision (k=3): 0.3180
[Epoch 60] Recall    (k=3): 0.2280
[Epoch 70] #original=1500, #synthesis=1500
[Epoch 70] Precision (k=3): 0.5673
[Epoch 70] Recall    (k=3): 0.3487
[Epoch 80] #original=1500, #synthesis=1500
[Epoch 80] Precision (k=3): 0.5080
[Epoch 80] Recall    (k=3): 0.4620
[Epoch 90] #original=1500, #synthesis=1500
[Epoch 90] Precision (k=3): 0.6113
[Epoch 90] Recall 